### image collect and store

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import os

name = input("Enter your name: ")

folder_path = os.path.join(os.getcwd(), name)
os.makedirs(folder_path, exist_ok=True)
print(f"Saving frames to: {folder_path}")

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

# Use both Pose and Hands (NO segmentation needed)
with mp_pose.Pose(static_image_mode=False,
                  model_complexity=1,
                  enable_segmentation=False,  # ✅ Segmentation turned off
                  min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose, \
     mp_hands.Hands(static_image_mode=False,
                    max_num_hands=2,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        # Step 1: Pose estimation
        pose_results = pose.process(image_rgb)

        # Step 2: Hand detection
        hand_results = hands.process(image_rgb)

        image_rgb.flags.writeable = True
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # We skip background removal and use original image directly
        output_image = image_bgr.copy()

        # Draw pose landmarks
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS)

        # Draw hand landmarks
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    output_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)

        # Save frame to folder
        filename = os.path.join(folder_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(filename, output_image)
        frame_count += 1

        # Show preview
        cv2.imshow('Pose + Hands (No Background Removed)', output_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Enter your name:  ankit


Saving frames to: E:\removeBackground\ankit


## Overwriting image in place from a directory

In [4]:
import os
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp
from pathlib import Path

# --- Get folder path ---
folder = "../Test_input"


def pillow_compress(image_path, quality):
    try:
        img = Image.open(image_path).convert("RGB")
        img.save(image_path, format='JPEG', quality=quality, optimize=True)
    except Exception as e:
        print(f"Compression error on {image_path}: {e}")

# --- Background removal using segmentation mask ---
def remove_background(image):
    h, w = image.shape[:2]
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True,
                      model_complexity=2,
                      enable_segmentation=True,
                      min_detection_confidence=0.05) as pose:

        results = pose.process(rgb_image)

        if not results.pose_landmarks:
            return image  # Return original if no person detected

        if results.segmentation_mask is not None:
            # Use Mediapipe segmentation
            mask = results.segmentation_mask > 0.001
            # Zero out background pixels (turn off)
            output = np.where(mask[..., None], image, 0).astype(np.uint8)
        else:
            # Use fallback convex hull + padding
            points = [(int(lm.x * w), int(lm.y * h)) for lm in results.pose_landmarks.landmark]
            hull = cv2.convexHull(np.array(points, dtype=np.int32))
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillConvexPoly(mask, hull, 255)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))  # ~2.5cm padding
            mask = cv2.dilate(mask, kernel, iterations=1)
            mask_3ch = cv2.merge([mask] * 3)
            output = np.where(mask_3ch == 255, image, 0).astype(np.uint8)

        return output

# --- Process all .jpg images ---
for filename in os.listdir(folder):
    if not filename.lower().endswith(".jpg"):
        continue

    image_path = os.path.join(folder, filename)
    print(f"Processing: {filename}")

    # Step 1: Initial compression (simulate Guetzli)
    pillow_compress(image_path, quality=65)

    original_img = cv2.imread(image_path)
    final_img = remove_background(original_img)
    cv2.imwrite(image_path, final_img)

    pillow_compress(image_path, quality=55)

print("✅ All images processed and replaced successfully.")

Processing: frame_0040.jpg


c:\DATA\Internships\Newro Kaaya\Test_01\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing: frame_0041.jpg
Processing: frame_0042.jpg
Processing: frame_0043.jpg
Processing: frame_0044.jpg
Processing: frame_0045.jpg
Processing: frame_0046.jpg
Processing: frame_0047.jpg
Processing: frame_0048.jpg
Processing: frame_0049.jpg
Processing: frame_0050.jpg
Processing: frame_0051.jpg
Processing: frame_0052.jpg
Processing: frame_0053.jpg
Processing: frame_0054.jpg
Processing: frame_0055.jpg
Processing: frame_0056.jpg
Processing: frame_0057.jpg
Processing: frame_0058.jpg
Processing: frame_0059.jpg
Processing: frame_0060.jpg
Processing: frame_0061.jpg
Processing: frame_0062.jpg
Processing: frame_0063.jpg
Processing: frame_0064.jpg
Processing: frame_0065.jpg
Processing: frame_0066.jpg
Processing: frame_0067.jpg
Processing: frame_0068.jpg
Processing: frame_0069.jpg
Processing: frame_0070.jpg
Processing: frame_0071.jpg
Processing: frame_0072.jpg
Processing: frame_0073.jpg
Processing: frame_0074.jpg
Processing: frame_0075.jpg
Processing: frame_0076.jpg
Processing: frame_0077.jpg
P

## Saving to a dedicated output folder

In [6]:
import os
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp
from pathlib import Path

# --- Input & Output folders ---
input_folder = "../Test_input"
output_folder = "imagecompression_out"

Path(output_folder).mkdir(parents=True, exist_ok=True)


def pillow_compress(image_path, quality):
    """Compress an image in place with Pillow at given quality."""
    try:
        img = Image.open(image_path).convert("RGB")
        img.save(image_path, format='JPEG', quality=quality, optimize=True)
    except Exception as e:
        print(f"Compression error on {image_path}: {e}")


# --- Background removal using segmentation mask ---
def remove_background(image):
    h, w = image.shape[:2]
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True,
                      model_complexity=2,
                      enable_segmentation=True,
                      min_detection_confidence=0.05) as pose:

        results = pose.process(rgb_image)

        if not results.pose_landmarks:
            return image  # Return original if no person detected

        if results.segmentation_mask is not None:
            # Use Mediapipe segmentation
            mask = results.segmentation_mask > 0.001
            output = np.where(mask[..., None], image, 0).astype(np.uint8)
        else:
            # Fallback: convex hull + padding
            points = [(int(lm.x * w), int(lm.y * h)) for lm in results.pose_landmarks.landmark]
            hull = cv2.convexHull(np.array(points, dtype=np.int32))
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillConvexPoly(mask, hull, 255)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
            mask = cv2.dilate(mask, kernel, iterations=1)
            mask_3ch = cv2.merge([mask] * 3)
            output = np.where(mask_3ch == 255, image, 0).astype(np.uint8)

        return output


# --- Process all .jpg images ---
for filename in os.listdir(input_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    print(f"Processing: {filename}")

    # Step 1: Initial compression (simulate Guetzli)
    pillow_compress(input_path, quality=65)

    original_img = cv2.imread(input_path)
    final_img = remove_background(original_img)

    cv2.imwrite(output_path, final_img)

    # Final compression on the saved output
    pillow_compress(output_path, quality=55)

print("✅ All images processed and saved to:", output_folder)


Processing: frame_0040.jpg
Processing: frame_0041.jpg
Processing: frame_0042.jpg
Processing: frame_0043.jpg
Processing: frame_0044.jpg
Processing: frame_0045.jpg
Processing: frame_0046.jpg
Processing: frame_0047.jpg
Processing: frame_0048.jpg
Processing: frame_0049.jpg
Processing: frame_0050.jpg
Processing: frame_0051.jpg
Processing: frame_0052.jpg
Processing: frame_0053.jpg
Processing: frame_0054.jpg
Processing: frame_0055.jpg
Processing: frame_0056.jpg
Processing: frame_0057.jpg
Processing: frame_0058.jpg
Processing: frame_0059.jpg
Processing: frame_0060.jpg
Processing: frame_0061.jpg
Processing: frame_0062.jpg
Processing: frame_0063.jpg
Processing: frame_0064.jpg
Processing: frame_0065.jpg
Processing: frame_0066.jpg
Processing: frame_0067.jpg
Processing: frame_0068.jpg
Processing: frame_0069.jpg
Processing: frame_0070.jpg
Processing: frame_0071.jpg
Processing: frame_0072.jpg
Processing: frame_0073.jpg
Processing: frame_0074.jpg
Processing: frame_0075.jpg
Processing: frame_0076.jpg
P